<a href="https://colab.research.google.com/github/Y0shicon/SOP/blob/main/SOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=j-A2E4OzjeF-osOLIKg4gwUvv9FhGik1uDa-TxLPr_8&tc=-QjQczY5QF2RJeTq8Yedcsuw756Q_Di7zV-tNCSXbQU&cc=IyJvKAnpXPZgR07C8caMfa_S7Se_YOFut3Kjvwo2hBc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AeaYSHD-wMhdn5MURFZtgJuEzOIq8DGAKJ2MDr2iDLbmRO_w-qSzYi7KUx4

Successfully saved authorization token.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
Map.setCenter(77.6, 28.5,9)

In [5]:
#location point
point=ee.Geometry.Point(77.6,28.5);

In [9]:
data = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(point)

In [10]:
image=ee.Image(data.filterDate("2019-01-01","2019-12-31").sort("CLOUD_COVERAGE_ASSESSMENT").first());

In [11]:
ndvi=image.expression(
"(NIR - RED) / (NIR + RED)",
{"NIR":image.select("B8"),
"RED":image.select("B4")});

In [12]:
display={
    "min":0,
    "max":1,
    "palette":[ 'red','orange', 'yellow','yellowgreen', 'green','black']
}

In [13]:
Map.addLayer(ndvi,display);
Map

Map(bottom=55003.0, center=[28.500074602152377, 77.59918212890626], controls=(WidgetControl(options=['position…